In [1]:
#library imports
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')

%matplotlib inline
import plotly
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as pyo
from plotly.subplots import make_subplots
import cufflinks as cf
from plotly.offline import init_notebook_mode,plot,iplot 

import folium # for maps

from zipfile import ZipFile 
import seaborn as sns
import requests 
from bs4 import BeautifulSoup 
#import geopandas as gpd
import os
import kaggle
from time import time,sleep
from random import randint
from IPython.core.display import clear_output
from requests import get
from bs4 import BeautifulSoup as soup
import matplotlib.pyplot as plt
import pandas as pd
from urllib.request import urlopen as uReq
import lxml.html as lh
import requests
from fbprophet import Prophet
import numpy as np
import json

In [2]:
pyo.init_notebook_mode(connected=True)
cf.go_offline()

In [3]:
total_cases_india=0
recover_cases_india=0
death_cases_india=0
active_cases_india=0

In [4]:
def getdata():
    url = 'https://www.mohfw.gov.in/' 

        # make a GET request to fetch the raw HTML content
    web_content = requests.get(url).content

        # parse the html content
    soup = BeautifulSoup(web_content, "html.parser")

        # remove any newlines and extra spaces from left and right
    extract_contents = lambda row: [x.text.replace('\n', '') for x in row] 

    stats = [] # initialize stats
    all_rows = soup.find_all('tr') # find all table rows 

    for row in all_rows: 
        stat = extract_contents(row.find_all('td')) # find all data cells  
            # notice that the data that we require is now a list of length 5
        if len(stat) == 5: 
            stats.append(stat)

        # now convert the data into a pandas dataframe for further processing
    new_cols = ["Sr.No", "States/UT","Confirmed","Recovered","Deceased"]
    state_data = pd.DataFrame(data = stats, columns = new_cols)
    state_data = state_data.iloc[:35,:]
    print(state_data)
    return state_data


In [5]:
def images():
    state_data = getdata()
    dec=[]

    for x in state_data['Deceased']:
        x = x.replace('#','')
        x = x.replace('*','')
        dec.append(x)
    state_data['Deceased']=dec
    dec.clear()

    for x in state_data['Confirmed']:
        x = x.replace('#','')
        x = x.replace('*','')
        dec.append(x)
    state_data['Confirmed']=dec
    dec.clear()

    for x in state_data['Recovered']:
        x = x.replace('#','')
        x = x.replace('*','')
        dec.append(x)
    state_data['Recovered']=dec
    dec.clear()

    for x in state_data['States/UT']:
        x = x.replace('#','')
        x = x.replace('*','')
        dec.append(x)
    state_data['States/UT']=dec
    dec.clear()


    # converting the 'string' data to 'int'
    state_data['Confirmed'] = state_data['Confirmed'].map(int)
    state_data['Recovered'] = state_data['Recovered'].map(int)
    state_data['Deceased']  = state_data['Deceased'].map(int)
    total_cases_india=state_data['Confirmed'].sum()
    recover_cases_india=state_data['Recovered'].sum()
    death_cases_india=state_data['Deceased'].sum()
    state_data['Active Cases']=state_data['Confirmed']-(state_data['Deceased']+state_data['Recovered'])
    active_cases_india = state_data['Active Cases'].sum()
    cases_in_India = [total_cases_india,recover_cases_india,death_cases_india]
    #plot1
    state_data.sort_values("Confirmed",ascending=False,inplace=True)
    fig = px.bar(state_data,x='States/UT',y='Confirmed',color='Confirmed',width=700, height=500,text='Confirmed',title="Confirmed Covid19 Cases in Indian States")
    fig.write_html("static/india_state_confirm_case.html")
    
    #plot2
    state_data.sort_values("Active Cases",ascending=False,inplace=True)
    fig = px.bar(state_data,x='States/UT',y='Active Cases',color='Active Cases',width=700,height = 500,text='Active Cases')
    fig.update_layout(title="Active Cases in Indian States")
    fig.write_html("static/india_state_active_case.html")
    
    
    #plot3
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=state_data['States/UT'], y=state_data['Confirmed'],mode='lines+markers',name='Confirmed Cases in each state',showlegend=True))
    fig.add_trace(go.Scatter(x=state_data['States/UT'], y=state_data['Deceased'],mode='lines+markers',name='Deaths Cases in each state',showlegend=True))
    fig.add_trace(go.Scatter(x=state_data['States/UT'], y=state_data['Recovered'],mode='lines+markers',name='Recovered Cases in each state',showlegend=True))
    fig.update_layout(title="All State Analysis",xaxis_title="States/UT",yaxis_title="Number of people")
    #fig.show()
    fig.write_html("static/india_states_analysis.html")
    
    #dataset
    f = open("indiageodata.json")
    counties = json.load(f)
    
    #plot4
    fig = px.choropleth_mapbox(state_data, geojson=counties, locations='States/UT', color='Active Cases',
                               color_continuous_scale="Viridis",featureidkey="properties.NAME_1",
                           mapbox_style="carto-positron",hover_data=["Confirmed", "Active Cases","Recovered","Deceased"],
                               zoom=3,center = {"lat": 20.5937, "lon": 78.9629},opacity=0.5,
                               labels={'Active Cases':'Active Cases'})
    fig.write_html('static/india_covidcases_map.html')
    
    
    #webscrap world corona cases
    import requests
    req = requests.get("https://covid.ourworldindata.org/data/owid-covid-data.csv")
    url_content = req.content
    csv_file = open(r'owid-covid-data.csv','wb')

    csv_file.write(url_content)
    csv_file.close()
    data = pd.read_csv(r'owid-covid-data.csv',parse_dates=True)
    data_italy = data[(data.location=='Italy')]
    data_us = data[(data.location=='United States')]
    data_india = data[(data.location=='India')]
    data_china = data[(data.location=='China')]
    
    
    #plot6
    fig=make_subplots(
    rows=2,cols=2,
    specs=[[{"secondary_y":True},{"secondary_y":True}],[{"secondary_y":True},{"secondary_y":True}]],
    subplot_titles=("China","Italy","United States","India"))

    fig.add_trace(go.Bar(x=data_china['date'],y=data_china['total_cases'],
                        marker=dict(color=data_china['total_cases'],coloraxis="coloraxis")),1,1)

    fig.add_trace(go.Bar(x=data_italy['date'],y=data_italy['total_cases'],
                        marker=dict(color=data_italy['total_cases'],coloraxis="coloraxis")),1,2)

    fig.add_trace(go.Bar(x=data_us['date'],y=data_us['total_cases'],
                        marker=dict(color=data_us['total_cases'],coloraxis="coloraxis")),2,1)

    fig.add_trace(go.Bar(x=data_india['date'],y=data_india['total_cases'],
                        marker=dict(color=data_india['total_cases'],coloraxis="coloraxis")),2,2)



    fig.update_layout(showlegend=False,title_text="Total Cases in 4 Countries",xaxis_title="Date",
        yaxis_title="Number of people")

    fig.update_layout(plot_bgcolor='rgb(169,169,169)')
    fig.write_html("static/country_comparision_total_cases.html")

    #data
    !kaggle datasets download -d sudalairajkumar/novel-corona-virus-2019-dataset --force
    file_name = "novel-corona-virus-2019-dataset.zip"
    # opening the zip file in READ mode 
    with ZipFile(file_name, 'r') as zip: 
        # extracting the file 
        zip.extract("covid_19_data.csv")
        zip.extract("time_series_covid_19_confirmed.csv")
    df=pd.read_csv(r'covid_19_data.csv',parse_dates=['Last Update'])
    df.rename(columns={'ObservationDate':'Date','Country/Region':'Country'},inplace=True)
    confirmed=df.groupby('Date').sum()['Confirmed'].reset_index()
    death=df.groupby('Date').sum()['Deaths'].reset_index()
    rec=df.groupby('Date').sum()['Recovered'].reset_index()
    
    #plot7
    fig=go.Figure()
    fig.add_trace(go.Scatter(x=confirmed['Date'],y=confirmed['Confirmed'],mode='lines+markers',name='Confirmed',line=dict(color='blue',width=2)))

    fig.add_trace(go.Scatter(x=death['Date'],y=death['Deaths'],mode='lines+markers',name='Deaths',line=dict(color='red',width=2)))
    fig.add_trace(go.Scatter(x=rec['Date'],y=rec['Recovered'],mode='lines+markers',name='Recovered',line=dict(color='green',width=2)))
    fig.update_layout(title_text="covid 19 cases in world",xaxis_title='Observation Date' ,yaxis_title='number of people')
    fig.write_html("static/world_confirm_death_cured.html")
    
    #data
    df_confirmed=pd.read_csv(r'time_series_covid_19_confirmed.csv')
    df_confirmed.rename(columns={'Country/Region':'Country'},inplace=True)
    df_latlong=pd.merge(df,df_confirmed,on=['Country','Province/State'])
    
    
    #plot9
    fig=px.scatter_mapbox(df_latlong,lat="Lat",lon="Long",zoom=1,height=700)
    fig.update_layout(title='Worldwide Corona Virus Cases')
    fig.update_layout(mapbox_style="carto-positron",mapbox_center_lon=0)
    fig.update_layout(margin={"r":2,"t":50,"l":2,"b":0},xaxis_title = "Observation Date",yaxis_title='Corona Cases')
    fig.write_html("static/world_cases_scatter_plot.html")
   
    return total_cases_india,recover_cases_india,death_cases_india,active_cases_india  

In [6]:
def predict():
  url = 'https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
  url_cnf = 'https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
  # for confirmed cases overall
  cnf = uReq(url_cnf)
  pg_cnf = requests.get(url_cnf)
  doc_cnf = lh.fromstring(pg_cnf.content)
  cnf_elements = doc_cnf.xpath('//tr')
  cnf.close()
  # for deaths overall
  client = uReq(url)
  page = requests.get(url)
  doc = lh.fromstring(page.content)
  tr_elements = doc.xpath('//tr')
  client.close()
  # for confirmed cases
  col_cnf=[]
  i=0
  #For each row, store each first element (header) and an empty list
  for t in cnf_elements[0]:
      i+=1
      name=t.text_content()
      #print ('%d:"%s"'%(i,name))
      col_cnf.append((name,[]))
  # for deaths
  col=[]
  i=0
  #For each row, store each first element (header) and an empty list
  for t in tr_elements[0]:
      i+=1
      name=t.text_content()
      #print ('%d:"%s"'%(i,name))
      col.append((name,[]))
  # for confirmed cases

  for j in range(1,len(cnf_elements)):
    T=cnf_elements[j]
    i=0
    for t in T.iterchildren():
          data=t.text_content() 
          if i>0:
          #Convert any numerical value to integers
              try:
                  data=int(data)
              except:
                  pass
          #Append the data to the empty list of the i'th column
          col_cnf[i][1].append(data)
          #Increment i for the next column
          i+=1
    # for deaths 

  for j in range(1,len(tr_elements)):
    T=tr_elements[j]
    i=0
    for t in T.iterchildren():
          data=t.text_content() 
          if i>0:
          #Convert any numerical value to integers
              try:
                  data=int(data)
              except:
                  pass
          #Append the data to the empty list of the i'th column
          col[i][1].append(data)
          #Increment i for the next column
          i+=1

  Dict={title:column for (title,column) in col}
  df=pd.DataFrame(Dict)
  d={title:column for (title,column) in col_cnf}
  cnf=pd.DataFrame(d)
  cnf.to_csv('time_series_cnf.csv')
  cnf.rename(columns={'Country/Region':'Country'}, inplace=True)
  col = ['Country','Lat','Long','Province/State']
  cnf_india = cnf[cnf['Country'] == 'India'].drop(col,axis = 1).transpose().reset_index().drop(0,axis = 0)

  cnf_india.rename(columns = {'index':'Date'}, inplace = True) 
  cnf_india.rename(columns = {131:'Confirmed'}, inplace = True) 
  fin_cnf = cnf_india
  index = fin_cnf.index
  length = len(index)
  for i in range(1,length+1):
    fin_cnf['Date'][i] = str(fin_cnf['Date'][i])
    fin_cnf['Confirmed'][i] = (fin_cnf['Confirmed'][i])
  fin_cnf.columns = ['ds','y']
  fin_cnf['ds'] = pd.to_datetime(fin_cnf['ds'])
  type(fin_cnf['ds'])
  confirmed = fin_cnf

  m = Prophet(interval_width=0.95,yearly_seasonality=True,daily_seasonality=True)
  m.fit(confirmed)
  future = m.make_future_dataframe(periods=7)
  future.tail()
  forecast = m.predict(future)
  forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

  df.rename(columns={'Country/Region':'Country'}, inplace=True)
  col = ['Country','Lat','Long','Province/State']
  df =  df[df['Country'] == 'India'].drop(col,axis = 1).transpose().reset_index().drop(0,axis = 0)

  df.rename(columns = {'index':'Date'}, inplace = True) 
  df.rename(columns = {131:'Death'}, inplace = True) 

  fin_df = df
  index = df.index
  length = len(index)
  for i in range(1,length+1):
    fin_df['Date'][i] = str(df['Date'][i])
    fin_df['Death'][i] = (df['Death'][i])

  fin_df.columns = ['ds','y']
  fin_df['ds'] = pd.to_datetime(fin_cnf['ds'])

  confirm = fin_df

  m = Prophet(interval_width=0.95,yearly_seasonality=True,daily_seasonality=True)
  m.fit(confirm)
  future_death = m.make_future_dataframe(periods=7)
  future_death.tail()
  forecast_death = m.predict(future)
  


  index = forecast_death.index
  length1 = len(index)

  death = []
  conf = []

  for i in range(length,length1):
    death.append(forecast_death['yhat_lower'][i])
    conf.append(forecast['yhat_lower'][i])
  
  forecast_death_7 = forecast_death.loc[length:len(forecast),['ds','yhat_lower']]
  forecast_cnf_7 = forecast.loc[length:len(forecast),['ds','yhat_lower']]
  forecast_death_7.to_csv('death_7.csv')
  forecast_cnf_7.to_csv('conf_7.csv')


In [7]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify, render_template
import pickle

In [8]:
total_cases_india,recover_cases_india,death_cases_india,active_cases_india = images()

   Sr.No                    States/UT Confirmed Recovered Deceased
0      1  Andaman and Nicobar Islands        33        33        0
1      2               Andhra Pradesh      3251      2125       59
2      3            Arunachal Pradesh         3         1        0
3      4                        Assam       856       104        4
4      5                        Bihar      3296      1211       15
5      6                   Chandigarh       288       189        4
6      7                 Chhattisgarh       399        83        0
7      8           Dadar Nagar Haveli         2         0        0
8      9                        Delhi     16281      7495      316
9     10                          Goa        69        38        0
10    11                      Gujarat     15562      8003      960
11    12                      Haryana      1504       881       19
12    13             Himachal Pradesh       276        70        5
13    14            Jammu and Kashmir      2036       859     


  0%|          | 0.00/1.11M [00:00<?, ?B/s]
 90%|########9 | 1.00M/1.11M [00:00<00:00, 5.26MB/s]
100%|##########| 1.11M/1.11M [00:00<00:00, 5.62MB/s]


In [9]:
predict()

In [10]:

death_pred = pd.read_csv("death_7.csv")
conf_pred = pd.read_csv("conf_7.csv")
conf_pred.drop(conf_pred.columns[[0]], axis = 1, inplace = True)
conf_pred.rename(columns = {"ds": "Date", "yhat_lower":"Predicted Cases"}, inplace = True)
death_pred.drop(death_pred.columns[[0]], axis = 1, inplace = True)
death_pred.rename(columns = {"ds": "Date", "yhat_lower":"Predicted Cases"}, inplace = True)
conf_pred['Predicted Cases'] = conf_pred['Predicted Cases'].map(int)
death_pred['Predicted Cases'] = death_pred['Predicted Cases'].map(int)

In [ ]:
from flask_ngrok import run_with_ngrok



from flask import Flask, request, jsonify, render_template
import pickle

app = Flask(__name__)
run_with_ngrok(app)




@app.route('/')
def home():
    return render_template('index.html',total_cases=total_cases_india, recovered_cases = recover_cases_india,death_cases=death_cases_india,active_cases=active_cases_india)

@app.route('/india')
def india():
    return render_template('index.html',total_cases=total_cases_india, recovered_cases = recover_cases_india,death_cases=death_cases_india,active_cases=active_cases_india)


@app.route('/worlddata')
def worlddata():
    return render_template('worldcase.html')


@app.route('/info')
def info():
    return render_template('info.html')

@app.route('/confirmcase')
def confirmcase():
    return render_template('predicted_confirm_cases.html',  tables=[conf_pred.to_html(classes='data')])

@app.route('/deathcase')
def deathcase():
    return render_template('predicted_death_cases.html',  tables=[death_pred.to_html(classes='data')])

@app.route('/helpline')
def helpline():
    return render_template('helpline.html')

@app.route('/twitter_analysis')
def twitter_analysis():
    return render_template('twitter analysis.html')


if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://8dde834c6894.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [29/May/2020 12:11:34] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/May/2020 12:11:34] "GET /static/default.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/May/2020 12:11:34] "GET /static/fonts.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/May/2020 12:11:34] "GET /static/images/COV19.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/May/2020 12:11:34] "GET /static/india_states_analysis.html HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/May/2020 12:11:34] "GET /static/india_state_confirm_case.html HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/May/2020 12:11:34] "GET /static/india_state_active_case.html HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/May/2020 12:11:36] "GET /static/india_covidcases_map.html HTTP/1.1" 200 -
